In [1]:
import argparse
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('image', cmap='gray')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import os
from EyeTracking import EyeTrackingDataset

In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--validation-percentage', type=float, default=15., metavar='P',
                   help='percentage of training data used for validation')
# parser.add_argument('--training-division', type=float, default=1., metavar='D',
#                    help='divide the remaining training data by this factor')
parser.add_argument('--epochs', type=int, default=14, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                    help='learning rate (default: 1.0)')
parser.add_argument('--step', type=int, default=1, metavar='N',
                    help='number of epochs between learning rate reductions (default: 1)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                    help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--reg-lambda', type=float, default=0.001, metavar='L',
                    help='Regularization lambda (default:0.001)')
parser.add_argument('--no-cuda', action='store_true',
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-numbers', type=int, default=10, metavar='N',
                    help='how many entries of logging training status to show per epoch')
parser.add_argument('--evaluate', action='store_true',
                    help='evaluate your model on the official test set')
parser.add_argument('--load-model', type=str,
                    help='model file path')
parser.add_argument('--save-model', type=str,
                    help='For Saving the current Model');

args = parser.parse_args('--validation-percentage 10 --batch-size 128 --epochs 8 --lr 1 --step 1 --gamma 0.9 --reg-lambda 0.0008 --seed 2020 --log-numbers 20'.split())

In [3]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
path_outputs = '../data/project'

In [4]:
dir_images = r'E:\Data\Unity\Minos\Fixation Training Images'
path_pos = r'E:\Data\Unity\Minos\Fixation Training Pos.bin'
dataset = EyeTrackingDataset(path_pos, dir_images, transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(60),
    #transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.ToTensor()
]))

In [5]:
rng = np.random.default_rng(args.seed)
idc = rng.permutation(len(dataset))
n_train = np.round(len(dataset)*(1-args.validation_percentage/100)).astype(int)

train_loader = DataLoader(
    dataset, batch_size=args.batch_size,
    sampler=SubsetRandomSampler(idc[:n_train]), **kwargs
)
val_loader = DataLoader(
    dataset, batch_size=args.batch_size,
    sampler=SubsetRandomSampler(idc[n_train:]), **kwargs
)

print(f'Loaded {len(dataset)} samples and divided into {len(train_loader.sampler)} training and {len(val_loader.sampler)} validation samples')

Loaded 64000 samples and divided into 57600 training and 6400 validation samples


In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.act = nn.ELU()
        # (1, 60, 160)
        
        self.d1 = 16
        self.conv1 = nn.Conv2d(1, self.d1, kernel_size=3, stride=1, padding=1, padding_mode='replicate')
        # (16, 60, 160)
        self.batchNorm1 = nn.BatchNorm2d(self.d1)
        self.pool1 = nn.MaxPool2d(2)
        self.dropout2d1 = nn.Dropout2d(0.8)
        # (16, 30, 80)
        
        self.d2 = 32
        self.conv2 = nn.Conv2d(self.d1, self.d2, kernel_size=3, stride=1, padding=1, padding_mode='replicate')
        # (32, 30, 80)
        self.batchNorm2 = nn.BatchNorm2d(self.d2)
        self.pool2 = nn.MaxPool2d(2)
        self.dropout2d2 = nn.Dropout2d(0.8)
        # (32, 15, 40)
        
        self.d3 = 64
        self.conv3 = nn.Conv2d(self.d2, self.d3, kernel_size=3, stride=1, padding=1, padding_mode='replicate')
        # (64, 15, 40)
        self.batchNorm3 = nn.BatchNorm2d(self.d3)
        self.pool3 = nn.MaxPool2d(2)
        self.dropout2d3 = nn.Dropout2d(0.8)
        # (64, 7, 20)
        
        self.d4 = 64
        self.conv4 = nn.Conv2d(self.d3, self.d4, kernel_size=3, stride=1, padding=1, padding_mode='replicate')
        # (64, 7, 20)
        self.batchNorm4 = nn.BatchNorm2d(self.d4)
        self.pool4 = nn.MaxPool2d(2)
        self.dropout2d4 = nn.Dropout2d(0.8)
        # (64, 3, 10)
        
        self.fc1 = nn.Linear(1920, 16)
        self.dropout1 = nn.Dropout(0.8)
        self.fc2 = nn.Linear(16, 16)
        self.dropout2 = nn.Dropout(0.8)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchNorm1(x)
        x = self.act(x)
        x = self.pool1(x)
        x = self.dropout2d1(x)

        x = self.conv2(x)
        x = self.batchNorm2(x)
        x = self.act(x)
        x = self.pool2(x)
        x = self.dropout2d2(x)

        x = self.conv3(x)
        x = self.batchNorm3(x)
        x = self.act(x)
        x = self.pool3(x)
        x = self.dropout2d3(x)

        x = self.conv4(x)
        x = self.batchNorm4(x)
        x = self.act(x)
        x = self.pool4(x)
        x = self.dropout2d4(x)

        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        #x = self.dropout1(x)
        
        x = self.fc2(x)
        x = self.act(x)
        #x = self.dropout2(x)
        
        x = self.fc3(x)
        
        return x

In [7]:
model = Net().to(device)
model.load_state_dict(torch.load(os.path.join(path_outputs, 'cnn1.pt')))

<All keys matched successfully>

In [8]:
model.eval()    # Set the model to inference mode
val_target = None
val_output = None
with torch.no_grad():   # For the inference step, gradient is not computed
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        if val_target is None:
            val_target = target.cpu().numpy()
        else:
            val_target = np.vstack((val_target, target.cpu().numpy()))
        if val_output is None:
            val_output = output.cpu().numpy()
        else:
            val_output = np.vstack((val_output, output.cpu().numpy()))

In [9]:
val_diff = val_output-val_target
val_result = np.dstack((val_target, val_output))
print(val_result.shape)

(6400, 3, 2)


In [12]:
%matplotlib widget
fig = plt.figure(figsize=(15, 10), tight_layout=True)
ax = fig.add_subplot(111, projection='3d')
for k in range(1000):
    ax.plot(val_result[k, 0, :], val_result[k, 1, :], val_result[k, 2, :])
ax.invert_yaxis()
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …